In [ ]:
# !pip install spotipy
# !pip install requests
# !pip install nltk


In [48]:
import re
import os
import spotipy
import requests
from spotipy.oauth2 import SpotifyClientCredentials
from bs4 import BeautifulSoup
import pandas as pd
from dotenv import load_dotenv

In [49]:
load_dotenv()

True

In [50]:
print()

patata


In [27]:
class GetLyrics():
    
    def __init__(self, spotify_client_id, spotify_client_secret, user_id, playlist_id, genius_key):
        self.spotify_client_id = os.getenv('SPOTIFY_CLIENT_ID')
        self.spotify_client_secret = os.getenv('SPOTIFY_CLIENT_ID')
        self.user_id = os.getenv('SPOTIFY_USER_ID')
        self.playlist_id = '5qU5rlfMhe9zfDLJgwemZk'
        self.genius_key = os.getenv('GENIUS_KEY') #we plugin personal spotfiy and genius credentials and playlist id
        
    def get_playlist_info(self):
        token = SpotifyClientCredentials(client_id=self.spotify_client_id,
                                         client_secret=self.spotify_client_secret).get_access_token()
        sp = spotipy.Spotify(token)
        playlist = sp.user_playlist_tracks(self.user_id, self.playlist_id)
        self.playlist = playlist
        return self.playlist
    
    def get_track_names(self):
        track_names = []
        for song in range(len(self.playlist['items'])):
            track_names.append(self.playlist['items'][song]['track']['name'])
        self.track_names = track_names
        return self.track_names
    
    def get_track_artists(self):
        track_artists = []
        for song in range(len(self.playlist['items'])):
            track_artists.append(self.playlist['items'][song]['track']['artists'][0]['name'])
        self.track_artists = track_artists
        return self.track_artists
        
    def request_song_info(self, track_name, track_artist):
        self.track_name = track_name
        self.track_artist = track_artist
        base_url = 'https://api.genius.com'
        headers = {'Authorization': 'Bearer ' + self.genius_key}
        search_url = base_url + '/search'
        data = {'q': track_name + ' ' + track_artist}
        response = requests.get(search_url, data=data, headers=headers)
        self.response = response
        return self.response

    def check_hits(self):
        json = self.response.json()
        remote_song_info = None
        for hit in json['response']['hits']:
            if self.track_artist.lower() in hit['result']['primary_artist']['name'].lower():
                remote_song_info = hit
                break
        self.remote_song_info = remote_song_info
        return self.remote_song_info
    
    def get_url(self):
        song_url = self.remote_song_info['result']['url']
        self.song_url = song_url
        return self.song_url
    
    def scrape_lyrics(self):
        page = requests.get(self.song_url)
        html = BeautifulSoup(page.text, 'html.parser')
        lyrics1 = html.find("div", class_="lyrics")
        lyrics2 = html.find("div", class_="Lyrics__Container-sc-1ynbvzw-2 jgQsqn")
        if lyrics1:
            lyrics = lyrics1.get_text()
        elif lyrics2:
            lyrics = lyrics2.get_text()
        elif lyrics1 == lyrics2 == None:
            lyrics = None
        return lyrics

    def get_lyrics(self):
        playlist = GetLyrics.get_playlist_info(self)
        track_names = GetLyrics.get_track_names(self)
        track_artists = GetLyrics.get_track_artists(self)
        song_lyrics = []
        for i in range(len(self.track_names)):
            print("\n")
            print(f"Working on track {i}.")
            response = GetLyrics.request_song_info(self, self.track_names[i], self.track_artists[i])
            remote_song_info = GetLyrics.check_hits(self)
            if remote_song_info == None:
                lyrics = None
                print(f"Track {i} is not in the Genius database.")
            else:
                url = GetLyrics.get_url(self)
                lyrics = GetLyrics.scrape_lyrics(self)
                if lyrics == None:
                    print(f"Track {i} is not in the Genius database.")
                else:
                    print(f"Retrieved track {i} lyrics!")
            song_lyrics.append(lyrics)
        return song_lyrics

In [28]:
songs = GetLyrics(os.getenv('SPOTIFY_CLIENT_ID'), os.getenv('SPOTIFY_CLIENT_SECRET'), os.getenv('SPOTIFY_USER_ID'), '5qU5rlfMhe9zfDLJgwemZk', os.getenv('GENIUS_KEY'))
song_lyrics = songs.get_lyrics() #this will get this the song lyrics using again credentials 



Working on track 0.
Retrieved track 0 lyrics!


Working on track 1.
Retrieved track 1 lyrics!


Working on track 2.
Retrieved track 2 lyrics!


Working on track 3.
Retrieved track 3 lyrics!


Working on track 4.
Retrieved track 4 lyrics!


Working on track 5.
Retrieved track 5 lyrics!


Working on track 6.
Retrieved track 6 lyrics!


Working on track 7.
Track 7 is not in the Genius database.


Working on track 8.
Track 8 is not in the Genius database.


Working on track 9.
Track 9 is not in the Genius database.


Working on track 10.
Retrieved track 10 lyrics!


Working on track 11.
Retrieved track 11 lyrics!


Working on track 12.
Track 12 is not in the Genius database.


Working on track 13.
Retrieved track 13 lyrics!


Working on track 14.
Retrieved track 14 lyrics!


Working on track 15.
Track 15 is not in the Genius database.


Working on track 16.
Retrieved track 16 lyrics!


Working on track 17.
Track 17 is not in the Genius database.


Working on track 18.
Retrieved track 18 

In [31]:
df = pd.DataFrame()
df['track_name'] = songs.track_names
df['artist'] = songs.track_artists
df['lyrics'] = song_lyrics 

In [33]:
df = df.dropna()
df = df.loc[df['lyrics'] != "[Non-Lyrical Vocals]"]
df = df.loc[df['lyrics'] != '']

In [34]:
df.reset_index(drop=True, inplace=True) #reset the index

In [35]:
song_lyrics = df['lyrics']

In [38]:
for i in range(len(song_lyrics)):
    song_lyrics[i] = re.sub("[\(\[].*?[\)\]]", " ", str(song_lyrics[i]))
    song_lyrics[i] = song_lyrics[i].replace("\n", " ")
    song_lyrics[i] = re.sub(r"(\w)([A-Z])", r"\1 \2", str(song_lyrics[i]))  #we iterate through the rows to delete punctuationan and other unnesary sympols

In [39]:
df

,track_name,artist,lyrics
0,AMERIKA,Rammstein,We're all living in Amerika Amerika ist...
1,Dream On,Aerosmith,Every time​ that I look in the mirror All the...
2,Enter Sandman,Metallica,"Say your prayers, little one, don't forget, ..."
3,Spanish Train,Chris de Burgh,There's a Spanish train that runs between ...
4,Whiskey In The Jar,Metallica,As I was going over the Cork and Kerry mounta...
5,Iron Man,Black Sabbath,I am Iron Man Has he lost his mind?Can he see...
6,Nothing Else Matters,Metallica,So close no matter how far Couldn't be ...
7,Paranoid,Black Sabbath,Finished with my woman 'cause she couldn't h...
8,Sad But True,Metallica,"Hey , I'm your life, I'm the one who takes y..."
9,Smells Like Teen Spirit,Nirvana,"Load up on guns, bring your friends It's f..."


In [41]:
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer

nltk.download('vader_lexicon')
sid = SentimentIntensityAnalyzer()

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/vangelistrikoupis/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [42]:
for i in range(len(df)):
    sentiment = sid.polarity_scores(df.loc[i, 'lyrics'])   #we get the polarity score for each row/song and thus iterate
    df.loc[i, 'neg'] = sentiment['neg']
    df.loc[i, 'neu'] = sentiment['neu']
    df.loc[i, 'pos'] = sentiment['pos']
    df.loc[i, 'compound'] = sentiment['compound']